In [1]:
#!pip install plotly
#!pip install yahoo_fin 
#!pip install pyomo

In [2]:
import numpy as np
import pandas as pd
import math 
import sys

from pyomo.environ import *
from pyomo.opt import SolverFactory

In [18]:
# List of stocks to use for data stream:
  # I'm using the QQQ portfolio for this project, however, users can replace the 
  # file with their own list of companies and build their own portfolio that way.
from yahoo_fin.stock_info import get_data

df = pd.read_excel('QQQ_portfolio.xlsx',header=None)
stocks = df[0].tolist()

data = {}
for ticker in stocks:
    data[ticker] = get_data(ticker, start_date="01/01/2010", end_date="01/01/2020", index_as_date = True, interval="1mo")
#data

In [15]:
# This data is stored as a dictionary of dataframse. 
# Looking at first few rows of the first stock in the list:
data['AAPL'].head()

,open,high,low,close,adjclose,volume,ticker
2010-01-01,7.622500,7.699643,6.794643,6.859286,5.880913,15168994400,AAPL
2010-02-01,6.870357,7.327500,6.816071,7.307857,6.265502,10776080000,AAPL
2010-03-01,7.348214,8.481429,7.337500,8.392857,7.195743,12154172800,AAPL
2010-04-01,8.478929,9.730714,8.312500,9.324643,7.994627,12367129600,AAPL
2010-05-01,9.422857,9.567143,7.116071,9.174286,7.865713,18082654800,AAPL


In [19]:
# Calculate monthly returns and add as a column
for i in stocks:
    for j in range(len(data[i])):
        data[i]['returns'] = data[i]['adjclose'].pct_change()  # this function is cited from https://www.codingfinance.com/post/2018-04-03-calc-returns-py/

# Drop the first month as it doesn't generate a return variable:
for i in stocks:
    data[i] = data[i].iloc[1: , :]

In [20]:
data['AAPL'].head()

,open,high,low,close,adjclose,volume,ticker,returns
2010-02-01,6.870357,7.327500,6.816071,7.307857,6.265501,10776080000,AAPL,0.065396
2010-03-01,7.348214,8.481429,7.337500,8.392857,7.195741,12154172800,AAPL,0.148470
2010-04-01,8.478929,9.730714,8.312500,9.324643,7.994621,12367129600,AAPL,0.111021
2010-05-01,9.422857,9.567143,7.116071,9.174286,7.865714,18082654800,AAPL,-0.016124
2010-06-01,9.274643,9.964643,8.650000,8.983214,7.701895,16651252800,AAPL,-0.020827


In [21]:
returns_list = [] 
for i in stocks:
    returns_list.append(data[i]['returns'].tolist())
        

In [22]:
# calculation of volatility for each stock:
  # first we need an average return for each stock: 
import statistics
avg_price = []
for i in stocks:
    avg_price.append(sum(data[i]['adjclose'])/len(data[i]))


In [23]:
#for i in stocks:
 #     data[i]['average'] = 0

for i in range(len(stocks)): 
      data[stocks[i]]['average'] = avg_price[i]
data['AAPL'].head()


,open,high,low,close,adjclose,volume,ticker,returns,average
2010-02-01,6.870357,7.327500,6.816071,7.307857,6.265501,10776080000,AAPL,0.065396,25.963639
2010-03-01,7.348214,8.481429,7.337500,8.392857,7.195741,12154172800,AAPL,0.148470,25.963639
2010-04-01,8.478929,9.730714,8.312500,9.324643,7.994621,12367129600,AAPL,0.111021,25.963639
2010-05-01,9.422857,9.567143,7.116071,9.174286,7.865714,18082654800,AAPL,-0.016124,25.963639
2010-06-01,9.274643,9.964643,8.650000,8.983214,7.701895,16651252800,AAPL,-0.020827,25.963639


In [24]:
for i in stocks:
    data[i]['devsqr'] = 0

for i in stocks:
    for j in range(len(data[i])):
        data[i]['devsqr'] = data[i]['average'] - data[i]['adjclose']


In [25]:
for i in stocks:
    data[i]['squared'] = 0

for i in stocks:
    for j in range(len(data[i])):
        data[i]['squared'][j] = data[i]['devsqr'][j]**2
        


<ipython-input-25-ae21021ab044>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[i]['squared'][j] = data[i]['devsqr'][j]**2


In [26]:
data['AAPL'].head()

,open,high,low,close,adjclose,volume,ticker,returns,average,devsqr,squared
2010-02-01,6.870357,7.327500,6.816071,7.307857,6.265501,10776080000,AAPL,0.065396,25.963639,19.698137,388
2010-03-01,7.348214,8.481429,7.337500,8.392857,7.195741,12154172800,AAPL,0.148470,25.963639,18.767897,352
2010-04-01,8.478929,9.730714,8.312500,9.324643,7.994621,12367129600,AAPL,0.111021,25.963639,17.969017,322
2010-05-01,9.422857,9.567143,7.116071,9.174286,7.865714,18082654800,AAPL,-0.016124,25.963639,18.097925,327
2010-06-01,9.274643,9.964643,8.650000,8.983214,7.701895,16651252800,AAPL,-0.020827,25.963639,18.261744,333


In [27]:
volatility = [] #indexed by i

for i in stocks:
    volatility.append( math.sqrt(data[i]['squared'].sum()/len(data[i])))


In [28]:
# Initial portfolio that minimizes volatility. 
    ## The portfolio returns are converted to annual gains, without including potential reinvestment of dividends
    
num_stocks = len(stocks) # indexed by i
num_obs = len(returns_list) # indexed by j, length of data stream

# Constructing the model
model = ConcreteModel()
model.x = Var(range(num_stocks), domain = NonNegativeReals) # this is the DV for % allocation to each stock.

# Objective: Minimize volatility  
model.Objective = Objective(expr = sum(model.x[i]*volatility[i] for i in range(num_stocks)), sense = minimize)


# Consraints
# Constraint 1 - portfolio allocations will sum up to 100% (or 1)
model.PortfolioConstraints = Constraint(expr = sum(model.x[i] for i in range(num_stocks)) == 1)

#model.pprint()


In [29]:
# Solve
opt = SolverFactory('glpk')
opt.solve(model), #tee=True

({'Problem': [{'Name': 'unknown', 'Lower bound': 1.53939855111003, 'Upper bound': 1.53939855111003, 'Number of objectives': 1, 'Number of constraints': 2, 'Number of variables': 103, 'Number of nonzeros': 103, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 0, 'Number of created subproblems': 0}}, 'Error rc': 0, 'Time': 0.04989504814147949}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]},)

In [30]:
# Solution list of tickers with portfolio allocations
print("Minimum Volatility is ", model.Objective()) # this is the minimum volatility from the given list of stocks
for i in range(num_stocks):
    if model.x[i]() != 0:
        print('stock',stocks[i], 'is allocated:', model.x[i]()*100,'%')
        print('Portfolio return is:', (sum(data[stocks[i]]['returns'])/num_obs)*100*12,'%')

Minimum Volatility is  1.5393985511100314
stock SIRI is allocated: 100.0 %
Portfolio return is: 31.03789251517206 %


In [31]:
# We would ideally like more returns, so we can add a minimum return constraint that any user can adjust as they wish
# First we need to delete the model (or make a brand new one)
model.del_component(model)


num_stocks = len(stocks) # indexed by i
num_obs = len(returns_list) # indexed by j, length of data stream

# Constructing the model
model = ConcreteModel()
model.x = Var(range(num_stocks), domain = NonNegativeReals) # this is the DV for % allocation to each stock.

# Objective: Minimize volatility  
model.Objective = Objective(expr = sum(model.x[i]*volatility[i] for i in range(num_stocks)), sense = minimize)


# Consraints
# Constraint 1 - portfolio allocations will sum up to 100% (or 1)
model.PortfolioConstraints = Constraint(expr = sum(model.x[i] for i in range(num_stocks)) == 1)
# Consrtaint 2 - portfolio's minimum return should be higher than QQQ
model.MinReturnConstraint = Constraint(expr = sum(sum(data[stocks[i]]['returns'])*model.x[i] for i in range(num_stocks))/num_obs >= 0.29/12)
    
#model.pprint()


In [32]:
# Solve
opt = SolverFactory('glpk')
opt.solve(model)#, tee=True)

{'Problem': [{'Name': 'unknown', 'Lower bound': 1.53939855111003, 'Upper bound': 1.53939855111003, 'Number of objectives': 1, 'Number of constraints': 3, 'Number of variables': 103, 'Number of nonzeros': 205, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 0, 'Number of created subproblems': 0}}, 'Error rc': 0, 'Time': 0.07327628135681152}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [33]:
print("Minimum Volatility is ", model.Objective()) # this is the minimum volatility from the given list of stocks
print("List of Stocks:")
for i in range(num_stocks):
    if model.x[i]() != 0:
        print('stock',stocks[i], 'is allocated:', model.x[i]()*100,'%')
        
list1 = []
for i in range(num_stocks):
    if model.x[i]() != 0:
        list1.append(sum(data[stocks[i]]['returns']*model.x[i]()))

      
print('Portfolio return is:', sum(list1)*12,'%')


Minimum Volatility is  1.5393985511100314
List of Stocks:
stock SIRI is allocated: 100.0 %
Portfolio return is: 31.658650365475502 %


In [34]:
# Next, I am adding another constraint for the allocation of the stock to make the portfolio more diversified
    # First we need to delete the model (or make a brand new one)
model.del_component(model)


num_stocks = len(stocks) # indexed by i
num_obs = len(returns_list) # indexed by j, length of data stream

# Constructing the model
model = ConcreteModel()
model.x = Var(range(num_stocks), domain = NonNegativeReals) # this is the DV for % allocation to each stock.

# Objective: Minimize volatility  
model.Objective = Objective(expr = sum(model.x[i]*volatility[i] for i in range(num_stocks)), sense = minimize)


# Consraints
# Constraint 1 - portfolio allocations will sum up to 100% (or 1)
model.PortfolioConstraints = Constraint(expr = sum(model.x[i] for i in range(num_stocks)) == 1)
# Consrtaint 2 - portfolio's minimum return should be higher than QQQ
model.MinReturnConstraint = Constraint(expr = sum(sum(data[stocks[i]]['returns'])*model.x[i] for i in range(num_stocks)) >= 0.29/12*num_obs)
# Constraint 3 - diversified portfolio
model.DiversifiedConstraint = ConstraintList()
for i in range(num_stocks):
    model.DiversifiedConstraint.add(expr = model.x[i] <= 0.5)
#model.pprint()

In [35]:
# Solve
opt = SolverFactory('glpk')
opt.solve(model)#, tee=True)

{'Problem': [{'Name': 'unknown', 'Lower bound': 2.68738414601473, 'Upper bound': 2.68738414601473, 'Number of objectives': 1, 'Number of constraints': 105, 'Number of variables': 103, 'Number of nonzeros': 307, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 0, 'Number of created subproblems': 0}}, 'Error rc': 0, 'Time': 0.05970454216003418}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [36]:
print("Minimum Volatility is ", model.Objective()) # this is the minimum volatility from the given list of stocks
print("List of Stocks:")
for i in range(num_stocks):
    if model.x[i]() != 0:
        print('stock',stocks[i], 'is allocated:', model.x[i]()*100,'%')
        
list1 = []
for i in range(num_stocks):
    if model.x[i]() != 0:
        list1.append(sum(data[stocks[i]]['returns']*model.x[i]()))

      
print('Portfolio return is:', sum(list1)*12,'%')


Minimum Volatility is  2.687384146014733
List of Stocks:
stock FOXA is allocated: 31.5536032229101 %
stock KDP is allocated: 18.446396777089898 %
stock SIRI is allocated: 50.0 %
Portfolio return is: 29.58 %


In [37]:
## From these results, we can see the tradeoff between a high return and a more diversified portfolio. 
## If we want to keep risk at minimum, we have to forego some of the returns.